In [ ]:
#count rows
import pandas as pd
chunksize = 1000000
i = 0

# 2019_3_deletesamezone.csv, 2019_5_add_duration.csv
for chunk in pd.read_csv('2019_5_add_duration.csv', chunksize = chunksize, index_col=0):
    #if (chunk['tpep_pickup_datetime'][1].any()):
        i = i + 1
        print(i)        
print('end')
print('Total number: ', i, 'Million rows')


In [5]:
# read data

import pandas as pd
from datetime import datetime
chunksize = 1000
i = 0

# 2018_Yellow_Taxi_Trip_Data.csv,  2018_1_columndelete.csv, 2018_2_complete.csv, 2018_3_deletesamezone.csv, 2018_4_calculate_costs.csv, 2018_5_add_duration.csv
# 2019_3_deletesamezone.csv
#for chunk in pd.read_csv('2019_2b_startafter6pm.csv', chunksize = chunksize, index_col=0):
for chunk in pd.read_csv('2018_Yellow_Taxi_Trip_Data.csv', chunksize = chunksize):
    
    print(chunk)
    print('--------------------------------------------------------')
    i += 1
    if (i>10):
        break

     VendorID    tpep_pickup_datetime   tpep_dropoff_datetime  \
0           2  05/09/2018 05:55:25 PM  05/09/2018 06:21:43 PM   
1           1  05/09/2018 05:00:14 PM  05/09/2018 05:34:05 PM   
2           1  05/09/2018 05:50:15 PM  05/09/2018 05:57:58 PM   
3           2  05/09/2018 05:55:58 PM  05/09/2018 06:20:17 PM   
4           2  05/09/2018 05:31:47 PM  05/09/2018 06:29:41 PM   
..        ...                     ...                     ...   
995         1  05/09/2018 05:54:36 PM  05/09/2018 06:40:53 PM   
996         2  05/09/2018 05:16:14 PM  05/09/2018 05:55:50 PM   
997         2  05/09/2018 06:00:22 PM  05/09/2018 06:31:32 PM   
998         1  05/09/2018 05:13:31 PM  05/09/2018 05:22:14 PM   
999         1  05/09/2018 05:23:10 PM  05/09/2018 05:42:05 PM   

     passenger_count  trip_distance  RatecodeID store_and_fwd_flag  \
0                  1           3.24           1                  N   
1                  2           3.40           1                  N   
2        

In [ ]:
# Drop unimportant columns

import pandas as pd
import os
if (os.path.isfile('2019_1_columndelete.csv')):
    os.remove('2019_1_columndelete.csv')
chunksize = 1000000
i = 0
header = True

for chunk in pd.read_csv('2019_Yellow_Taxi_Trip_Data.csv', chunksize = chunksize, index_col=0):
    i = i+1
    print(i)
    #drop vendorID, passenger_count, extra, payment_type ..
    chunk = chunk.drop(columns=['VendorID','passenger_count','RatecodeID','store_and_fwd_flag','payment_type','fare_amount', 'extra','mta_tax','tolls_amount','improvement_surcharge','congestion_surcharge'])
    chunk.to_csv('2019_1columndelete.csv', mode = 'a', header = header)
    header = False
    
    #print(chunk)
print('fertig')


In [ ]:
# Compare Datetime all-in-one + save right timewindow in new csv (Dateformat for 2015)
import pandas as pd
from datetime import datetime
import os
if (os.path.isfile('yellow_tripdata_2015-01-06_6to6.csv')):
    os.remove('yellow_tripdata_2015-01-06_6to6.csv')
chunksize = 10000
i = 0
header = True

compare_am = datetime.strptime('06:00:00', '%H:%M:%S').time()
compare_pm = datetime.strptime('18:00:00', '%H:%M:%S').time()

for chunk in pd.read_csv('2016_Yellow_Taxi_Trip_Data.csv', chunksize = chunksize):
    counter = 0
    print(i)
    while (counter < chunksize):
        string_pickup = chunk['tpep_pickup_datetime'][i]
        datetime_pickup = datetime.strptime(string_pickup,'%Y-%m-%d %H:%M:%S')
        string_dropoff = chunk['tpep_dropoff_datetime'][i]
        datetime_dropoff = datetime.strptime(string_dropoff,'%Y-%m-%d %H:%M:%S')
            
        # check if before 6am and after 6 pm
        help_time_start = datetime_pickup.time()
        help_time_end = datetime_dropoff.time()
            
        # Check if time is in time-window
        #if (compare_am > help_time_start or compare_pm < help_time_start or compare_am > help_time_end or compare_pm < help_time_end):
        if (compare_am > help_time_start):        
            chunk = chunk.drop([i])                
            
        i =  i + 1
        counter = counter + 1
    chunk.to_csv('yellow_tripdata_2015-01-06_6to6.csv', mode = 'a', header = header)
    header = False
    #print(chunk)
    #print('--------------------------------------------------------')
print('finished')

In [ ]:
#to check if row deletion was correct

import pandas as pd
from datetime import datetime
chunksize = 10000
i = 0

compare_am = datetime.strptime('06:00:00', '%H:%M:%S').time()
compare_pm = datetime.strptime('18:00:00', '%H:%M:%S').time()


for chunk in pd.read_csv('yellow_tripdata_2015-01-06_6to6.csv', chunksize = chunksize):
    counter = 0
    
    while (counter < chunksize):
        if (chunk['tpep_pickup_datetime'][i]):
            string_pickup = chunk['tpep_pickup_datetime'][i]
            datetime_pickup = datetime.strptime(string_pickup,'%Y-%m-%d %H:%M:%S')
            string_dropoff = chunk['tpep_dropoff_datetime'][i]
            datetime_dropoff = datetime.strptime(string_dropoff,'%Y-%m-%d %H:%M:%S')
            difference = datetime_dropoff - datetime_pickup
            
            # check if before 6am and after 6 pm
            help_time_start = datetime_pickup.time()
            help_time_end = datetime_dropoff.time()
            
            
            # The comparison here is I think not as expected the latest time is the smallest in the coparison
            if (compare_am > help_time_start or compare_pm < help_time_start or compare_am > help_time_end or compare_pm < help_time_end):
                window = False
            else:
                window = True
                
            
            print(string_pickup, string_dropoff, 'Difference: ', difference, 'Time-window ', help_time_start, window)
            i =  i + 1
            counter = counter + 1
    
    #print(chunk)
    #print('--------------------------------------------------------')
print('finished 2 complety')

In [ ]:
# Combine Files

import pandas as pd

CHUNK_SIZE = 50000
csv_file_list = ["file1.csv", "file2.csv", "file3.csv"]
output_file = "./result_merge/output.csv"

first_one = True
for csv_file_name in csv_file_list:

    if not first_one: # if it is not the first csv file then skip the header row (row 0) of that file
        skip_row = [0]
    else:
        skip_row = []

    chunk_container = pd.read_csv(csv_file_name, chunksize=CHUNK_SIZE, skiprows = skip_row)
    for chunk in chunk_container:
        chunk.to_csv(output_file, mode="a", index=False)
    first_one = False

### Todo
delete unnecessary columns
<br>
add areas to files that need it<br>
(add the triptime)<br>
check if the columns are the same + in the same order<br>
merge files<br>

In [ ]:
import numpy as np
filedata = np.genfromtxt('matrix00_edited.txt', delimiter = ' ')
print(filedata)

In [ ]:
# Create shorter file for testing purposes
import pandas as pd
import os
os.remove('short_tripdata.csv')
chunksize = 1000
counter = 0
header = True
for chunk in pd.read_csv('yellow_tripdata_2015-01-06.csv', chunksize = chunksize):
    if (counter < 1):
        chunk.to_csv('short_tripdata.csv', mode = 'a', header = header)
        counter = counter + 1
        header = False
    else:
        break
    print('--------------------------------------------------------')

In [ ]:
import pandas as pd
from datetime import datetime
import os


# 1 Drop unimportant columns

def function_1(year):
    
    chunksize = 1000
    i = 0
    header = True
    if (os.path.isfile(year + '_1_columndelete.csv')):
        os.remove(year + '_1_columndelete.csv')


    for chunk in pd.read_csv(year + '_Yellow_Taxi_Trip_Data.csv', chunksize = chunksize):
        i = i+1
        print(i)
        #drop vendorID, passenger_count, extra, payment_type ..
        #chunk = chunk.drop(columns=['VendorID','passenger_count','RatecodeID','store_and_fwd_flag','payment_type','fare_amount', 'extra','mta_tax','tolls_amount','improvement_surcharge','congestion_surcharge'])
        chunk[['tpep_pickup_datetime','tpep_dropoff_datetime','trip_distance','PULocationID','DOLocationID','tip_amount','total_amount']].to_csv(year + '_1_columndelete.csv', mode = 'a', header = header, index = False)
        header = False

    print('finished 1')

function_1('2018')